In [737]:
import requests
from datetime import timedelta
import json
import pandas as pd
import plotly.express as px
from pandas import json_normalize  

url = "https://rootd4.vercel.app/score/all"
response = requests.get(url)
dados = response.json()
# Transformar a resposta JSON em DataFrame
df = pd.DataFrame(response.json()).copy()


In [738]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_colwidth', 150)
pd.set_option('display.width', 120)
pd.set_option('expand_frame_repr', True)

## Quantidade de partidas

In [739]:
df.shape[0]

165

## ID Pokemon

In [740]:
def support_id_nome(id):
    support_names = {
        0: "Arcanine",
        1: "Azumarill",
        2: "Flareon",
        3: "Kingler",
        4: "Leafeon",
        5: "Glalie",
        6: "Beedrill",
        7: "Vaporeon",
        8: "Electabuzz",
        9: "Ampharos",
        10: "Muk",
        11: "Jolteon",
        12: "Rhydon",
        13 : "MrMime",
        14: "Swampert",
        15: "Glaceon",
        16: "Alakazam",
        17: "Pidgeot",
        18: "Meganium",
        19: "Umbreon",
        20: "Victribell",
        21: "Espeon",
        22: "Magneton"
    }
    return support_names.get(int(id), "Unknown Support")


def hero_nome_id(id):
    hero_names = {
        0: "Bulbasaur",
        1: "Charmander",
        2: "Squirtle",
        3: "Pikachu"
    }
    return hero_names.get(int(id), "Unknown Hero")

In [741]:
df['info'] = df['info'].apply(json.loads)

In [742]:
# Normalizar 'info'
info_df = pd.json_normalize(df['info'].tolist())

# Adicionar 'name' em info
info_df['name'] = df['name']
info_df['time'] = pd.to_numeric(df['time'])

# Filtrar registros com pokemon diferente de -1
stats_df = pd.json_normalize(df['info'].tolist(), 'statistics')
stats_df['pokemon'] = stats_df['pokemon'].astype(int)

# Adicionar 'name' em stats
stats_df['name'] = df['name'].repeat(info_df['statistics'].apply(len).tolist()).reset_index(drop=True)

stats_dfo = stats_df
stats_df = stats_df.query('pokemon != -1')

## TOP 5 Suportes usados por names

In [743]:
top5_suportes_jogadores = info_df

stats_df = stats_df.query('pokemon != -1')

top5_suportes_jogadores = (
    stats_df.groupby(['name', 'pokemon'])
    .size()
    .reset_index(name='count')
    .sort_values(by=['name', 'count'], ascending=[True, False])
    .groupby('name', as_index=False)  # Evita a criação de índice adicional
    .head(5)
    .groupby('name')['pokemon']
    .agg(lambda x: ', '.join(map(str, x)))
    .reset_index(name='Top 5 Suportes')
)

# Adicionar nomes de Pokémon usando a função support_id_nome
top5_suportes_jogadores['Top 5 Suportes'] = top5_suportes_jogadores['Top 5 Suportes'].apply(lambda x: ', '.join(map(support_id_nome, x.split(','))))


# Exibir o DataFrame resultante
top5_suportes_jogadores[['name', 'Top 5 Suportes']]



,name,Top 5 Suportes
0,Delirio,"Arcanine, Muk, Jolteon, MrMime, Glalie"
1,Guiltybr,"Arcanine, Kingler, Glalie, Electabuzz, Muk"
2,Jihn,"Electabuzz, Umbreon, Magneton, Arcanine, Kingler"
3,Jumbrado,"Arcanine, Beedrill, Electabuzz, Kingler, Muk"
4,KNOCK,"Arcanine, Kingler, Ampharos, Muk, Jolteon"
5,Ks,"Arcanine, Meganium, Kingler, Beedrill, Muk"
6,Lipekill,"Arcanine, Magneton, Kingler, Electabuzz, Muk"
7,Lironi,"Arcanine, Electabuzz, Kingler, Glaceon, Meganium"
8,Luhmb,"Kingler, Beedrill, Electabuzz, Alakazam, Magneton"
9,MatthawsRdC,"Jolteon, MrMime, Espeon, Glalie, Vaporeon"


## Suportes mais utilizados

In [744]:
# Contar a ocorrência de cada Pokémon
suportes_mais_usados = stats_df['pokemon'].value_counts().reset_index(name='count')

#Remover o heroi -1
suportes_mais_usados = suportes_mais_usados.query('pokemon != -1').reset_index(drop=True)

# Adicionar nomes de Pokémon usando a função support_id_nome
suportes_mais_usados['pokemon'] = suportes_mais_usados['pokemon'].apply(support_id_nome)

# Criar o gráfico de barras com o Plotly
fig = px.bar(suportes_mais_usados, x='pokemon', y='count', title='Pokémons de Suporte Mais Usados')
fig.update_layout(xaxis=dict(tickangle=45))  # Ajustar a rotação dos rótulos para melhor legibilidade

# Exibir o gráfico
#fig.show()

suportes_mais_usados = suportes_mais_usados.rename(columns={'pokemon': 'Nome', 'count':'Quant'})
suportes_mais_usados


,Nome,Quant
0,Arcanine,121
1,Kingler,101
2,Electabuzz,81
3,Beedrill,67
4,Meganium,63
5,Umbreon,62
6,Magneton,42
7,Azumarill,40
8,Muk,34
9,Ampharos,33


## Maior tempo de jogo

In [745]:
# Adicione esta linha
aux_df = info_df

# Calcular a quantidade total de tempo jogado por 'name' em segundos
tempo_total_jogo_nome = aux_df.groupby('name')['time'].sum().reset_index(name='tempo_total_segundos')

# Converter os segundos para horas
tempo_total_jogo_nome['tempo_total_horas'] = tempo_total_jogo_nome['tempo_total_segundos'] / 3600

# Converter os segundos para formato de hora
tempo_total_jogo_nome['tempo_total_horas'] = pd.to_datetime(tempo_total_jogo_nome['tempo_total_segundos'], unit='s').dt.strftime('%H:%M:%S')

# Exibir a tabela
#tempo_total_jogo_nome[['name', 'tempo_total_horas']]

# Criar um gráfico de barras interativo com o Plotly
fig = px.bar(
    tempo_total_jogo_nome,
    x='name',
    y='tempo_total_horas',
    labels={'total_time_seconds': 'Tempo Total Jogado (segundos)'},
    title='Distribuição do Tempo Total Jogado por Jogador',
)
tempo_total_jogo_nome['tempo_total_horas'] = pd.to_timedelta(tempo_total_jogo_nome['tempo_total_horas'])

media_tempo = tempo_total_jogo_nome['tempo_total_horas'].mean()





# Exibir o gráfico
fig.show()

## Jogadores Vitórias

In [746]:
# Criar uma tabela com a contagem de cada jogador
vitorias_jogadores = info_df.groupby('name').size().reset_index(name='vitorias').sort_values(by='vitorias', ascending=False)

# Converter a coluna 'time' para numérica
info_df['time'] = pd.to_numeric(info_df['time'], errors='coerce')

# Calcular a média de tempo para cada jogador em minutos
media_tempo = info_df.groupby('name')['time'].mean().reset_index(name='media_tempo')
media_tempo['media_tempo'] /= 60  # Convertendo para minutos
media_tempo['media_tempo'] = pd.to_datetime(media_tempo['media_tempo'], unit='m').dt.strftime('%M:%S.%f')

# Exibir a tabela
resultado_final = pd.merge(vitorias_jogadores, media_tempo, on='name')

resultado_final



,name,vitorias,media_tempo
0,Nico,30,23:34.903861
1,Xanad0,29,23:02.112898
2,kawa,21,23:47.180623
3,Royal,18,24:08.372226
4,NewJeans,9,24:06.641777
5,Jumbrado,8,24:10.686492
6,Weiss,7,27:30.258562
7,Ks,6,26:34.745015
8,Mestre LpK,6,37:07.591488
9,ftbr,5,26:19.986206


## Quantidade jogada heróis

In [747]:
# Criar um DataFrame com os dados
contagem_heroi = info_df

contagem_heroi['hero_nome'] = contagem_heroi['character'].apply(hero_nome_id)

# Criar uma tabela com a contagem de cada personagem usando groupby
contagem_heroi = info_df.groupby(['hero_nome']).size().reset_index(name='quantidade')

# Exibir o gráfico
#contagem_heroi

# Criar um gráfico de pizza com Plotly Express
fig = px.pie(contagem_heroi, values='quantidade', names='hero_nome', title='Contagem de Personagens por Nome de Herói')

fig.show()


## Herói mais jogado por jogador

In [748]:
heroi_preferido_por_jogador = info_df 

# Calcular a contagem de ocorrências de cada personagem por jogador
heroi_preferido_por_jogador = heroi_preferido_por_jogador.groupby(['name', 'hero_nome']).size().reset_index(name='quantidade')

# Encontrar o índice do primeiro valor máximo para cada jogador
idx = heroi_preferido_por_jogador.groupby('name')['quantidade'].idxmax()
heroi_preferido_por_jogador = heroi_preferido_por_jogador.loc[idx]

# Exibir a tabela final
heroi_preferido_por_jogador[['name', 'hero_nome', 'quantidade']]


,name,hero_nome,quantidade
0,Delirio,Bulbasaur,1
2,Guiltybr,Bulbasaur,1
4,Jihn,Pikachu,2
5,Jumbrado,Bulbasaur,4
8,KNOCK,Bulbasaur,4
9,Ks,Bulbasaur,5
11,Lipekill,Bulbasaur,1
13,Lironi,Bulbasaur,2
15,Luhmb,Bulbasaur,1
16,MatthawsRdC,Pikachu,2


## Suporte mais jogado pelos jogadores

In [749]:
suporte_preferido_por_jogador = stats_df

# Adicionar uma coluna com os nomes dos Heróis usando a função
suporte_preferido_por_jogador['support_nome'] = suporte_preferido_por_jogador['pokemon'].apply(support_id_nome)

suporte_preferido_por_jogador = suporte_preferido_por_jogador.groupby(['name', 'support_nome']).size().reset_index(name='quantidade')

# Exibir as primeiras linhas do DataFrame normalizado
idx = suporte_preferido_por_jogador.groupby('name')['support_nome'].idxmax()
suporte_preferido_por_jogador = suporte_preferido_por_jogador.loc[idx]

suporte_preferido_por_jogador[['name', 'support_nome', 'quantidade']]

,name,support_nome,quantidade
5,Delirio,Muk,2
15,Guiltybr,Umbreon,1
22,Jihn,Umbreon,2
33,Jumbrado,Umbreon,1
45,KNOCK,Vaporeon,1
59,Ks,Victribell,1
67,Lipekill,Victribell,1
79,Lironi,Swampert,1
84,Luhmb,Magneton,1
91,MatthawsRdC,Victribell,1


# DPS

## Geral

In [750]:
dps_geral = stats_dfo.copy()

dps_geral['nome'] = dps_geral['pokemon'].apply(support_id_nome)

# Adicionar colunas 'name' e 'time' de volta
dps_geral['time'] = df['time'].repeat(info_df['statistics'].apply(len).tolist()).reset_index(drop=True)

# Converter colunas para float
dps_geral['startTime'] = dps_geral['startTime'].str.replace(',', '.').astype(float)
dps_geral['time'] = dps_geral['time'].str.replace(',', '.').astype(float)
dps_geral['damage'] = dps_geral['damage'].str.replace(',', '.').astype(int)

# Calcular DPS
dps_geral['dps'] = dps_geral['damage'] / (dps_geral['time'] - dps_geral['startTime'])

# Calcular média do DPS para cada pokemon
dps_media = dps_geral.groupby(['nome', 'pokemon'])['dps'].mean().reset_index()


## DPS de cada suporte

In [751]:
# Filtrar registros com pokemon diferente de -1
dps_media_suporte = dps_media.query('pokemon != -1')

# Criar gráfico interativo com Plotly Express
fig = px.bar(
    dps_media_suporte,
    x='nome',
    y='dps',
    labels={'avg_dps': 'DPS Médio', 'pokemon_name': 'Suporte'},
)

# Ajustar layout para melhor visualização
#fig.update_layout(xaxis_tickangle=-45)

#Exibir o gráfico
#fig.show()

## Dps Heroi

In [752]:
stats_df = dps_geral.copy()

dps_media_heroi = stats_df.query('pokemon == -1').reset_index(drop=True)

# Adicionar coluna 'hero' com os IDs dos heróis repetidos
dps_media_heroi['hero'] = info_df['character'].repeat(info_df['statistics'].apply(len).tolist()).reset_index(drop=True)

# Adicionar coluna 'hero_nome' com os nomes correspondentes aos heróis
dps_media_heroi['hero_nome'] = info_df['character'].apply(hero_nome_id)

# Calcular a média do DPS para cada herói
dps_media_heroi = dps_media_heroi.groupby('hero_nome')['dps'].mean().reset_index()

# Renomear a coluna do índice para 'hero_nome'
dps_media_heroi = dps_media_heroi.rename(columns={'hero_nome': 'hero_nome'})

# Exibir o DataFrame resultante
dps_media_heroi

,hero_nome,dps
0,Bulbasaur,194.578523
1,Charmander,148.604739
2,Pikachu,191.911764
3,Squirtle,149.689138


In [753]:
info_df_id_partida = info_df.copy()

# Adicionar uma nova coluna 'id_partida' que representa o ID único para cada linha
info_df_id_partida['id_partida'] = info_df_id_partida.index + 1 

# Filtrar linhas da coluna 'pokemon'
stats_df_id_partidah = stats_df.query('pokemon == -1').reset_index(drop=True)

# Adicionar coluna 'hero' e 'id_partida' com os IDs dos heróis repetidos
stats_df_id_partidah['hero'] = info_df_id_partida['character'].reset_index(drop=True)
stats_df_id_partidah['id_partida'] = info_df_id_partida['id_partida'].reset_index(drop=True)

# Adicionar coluna 'hero_nome' com os nomes correspondentes aos heróis
stats_df_id_partidah['nome'] = stats_df_id_partidah['hero'].apply(hero_nome_id)


stats_df_id_partidas = stats_df.copy()

stats_df_id_partidas['id_partida'] = info_df_id_partida['id_partida'].repeat(info_df_id_partida['statistics'].apply(len).tolist()).reset_index(drop=True)

# Filtrar linhas da coluna 'pokemon'
stats_df_id_partidas = stats_df_id_partidas.query('pokemon != -1').reset_index(drop=True)

stats_df_id_partidas



,startTime,pokemon,damage,name,nome,time,dps,id_partida
0,122.7551,3,12869,Xanad0,Kingler,1280.947998,11.111275,1
1,217.8720,0,21683,Xanad0,Arcanine,1280.947998,20.396472,1
2,435.9753,19,24401,Xanad0,Umbreon,1280.947998,28.877856,1
3,806.3627,6,16531,Xanad0,Beedrill,1280.947998,34.832516,1
4,991.2443,8,9001,Xanad0,Electabuzz,1280.947998,31.069676,1
...,...,...,...,...,...,...,...,...
820,221.9673,4,43221,Mestre LpK,Leafeon,3599.999002,12.794729,165
821,690.0281,16,77111,Mestre LpK,Alakazam,3599.999002,26.498890,165
822,1501.2180,11,75893,Mestre LpK,Jolteon,3599.999002,36.160514,165
823,3171.3830,13,0,Mestre LpK,MrMime,3599.999002,0.000000,165


In [754]:
info_df

,character,level,statistics,endTime,nomePc,name,time,hero_nome
0,3,32,"[{'startTime': '0,000000', 'pokemon': '-1', 'damage': '252072'}, {'startTime': '122,755100', 'pokemon': '3', 'damage': '12869'}, {'startTime': '21...",21:20.948,DESKTOP-K7TGG6S,Xanad0,1280.947998,Pikachu
1,1,31,"[{'startTime': '0,000000', 'pokemon': '-1', 'damage': '199981'}, {'startTime': '123,868600', 'pokemon': '3', 'damage': '39379'}, {'startTime': '21...",21:48.080,DESKTOP-P511SQ8,Nico,1308.079956,Charmander
2,1,33,"[{'startTime': '0.000000', 'pokemon': '-1', 'damage': '195169'}, {'startTime': '114.023200', 'pokemon': '3', 'damage': '41197'}, {'startTime': '22...",21:53.788,DESKTOP-QBR207K,kawa,1313.787964,Charmander
3,1,33,"[{'startTime': '0,000000', 'pokemon': '-1', 'damage': '200447'}, {'startTime': '113,977500', 'pokemon': '19', 'damage': '39939'}, {'startTime': '2...",21:55.124,DESKTOP-K7TGG6S,Xanad0,1315.124023,Charmander
4,1,32,"[{'startTime': '0,000000', 'pokemon': '-1', 'damage': '191340'}, {'startTime': '118,640800', 'pokemon': '18', 'damage': '38808'}, {'startTime': '2...",21:56.333,DESKTOP-P511SQ8,Nico,1316.333008,Charmander
...,...,...,...,...,...,...,...,...
160,1,32,"[{'startTime': '0,000000', 'pokemon': '-1', 'damage': '248724'}, {'startTime': '137,892200', 'pokemon': '11', 'damage': '19257'}, {'startTime': '3...",36:55.494,DESKTOP-VO0R3J4,Mestre LpK,2215.493896,Charmander
161,3,31,"[{'startTime': '0,000953', 'pokemon': '-1', 'damage': '317870'}, {'startTime': '223,391100', 'pokemon': '7', 'damage': '6995'}, {'startTime': '488...",37:47.416,DESKTOP-P511SQ8,Nico,2267.416016,Pikachu
162,0,36,"[{'startTime': '0,000000', 'pokemon': '-1', 'damage': '375407'}, {'startTime': '154,060100', 'pokemon': '19', 'damage': '20579'}, {'startTime': '2...",37:50.273,DESKTOP-IDREH82,ackloh mito,2270.272949,Bulbasaur
163,0,37,"[{'startTime': '0,000000', 'pokemon': '-1', 'damage': '478721'}, {'startTime': '145,042400', 'pokemon': '16', 'damage': '30000'}, {'startTime': '4...",41:37.698,DESKTOP-PNMRL5A,Paula,2497.697998,Bulbasaur


## Ranking Geral

In [759]:
info_df_exp = info_df.copy()
# Normalizar a coluna 'statistics'

info_df_exp = json_normalize(info_df['statistics'].tolist())

# Juntar o DataFrame normalizado de volta ao DataFrame original
info_df_exp = pd.concat([info_df, info_df_exp], axis=1)

# Descartar a coluna 'statistics' original se desejado
info_df_exp = info_df_exp.drop('statistics', axis=1)

# Abrir tabelas
info_df_exp0 = info_df_exp[0].apply(pd.Series).rename(columns={'startTime': 'startTime0', 'pokemon': 'pokemon0', 'damage': 'damage0'})
info_df_exp1 = info_df_exp[1].apply(pd.Series).rename(columns={'startTime': 'startTime1', 'pokemon': 'pokemon1', 'damage': 'damage1'})
info_df_exp2 = info_df_exp[2].apply(pd.Series).rename(columns={'startTime': 'startTime2', 'pokemon': 'pokemon2', 'damage': 'damage2'})
info_df_exp3 = info_df_exp[3].apply(pd.Series).rename(columns={'startTime': 'startTime3', 'pokemon': 'pokemon3', 'damage': 'damage3'})
info_df_exp4 = info_df_exp[4].apply(pd.Series).rename(columns={'startTime': 'startTime4', 'pokemon': 'pokemon4', 'damage': 'damage4'})
info_df_exp5 = info_df_exp[5].apply(pd.Series).rename(columns={'startTime': 'startTime5', 'pokemon': 'pokemon5', 'damage': 'damage5'})

info_df_exp = pd.concat([info_df_exp, info_df_exp0, info_df_exp1, info_df_exp2, info_df_exp3, info_df_exp4, info_df_exp5], axis=1)

# Detalhamento da tabela
info_df_exp['pokemon0'] = info_df_exp['character'].apply(hero_nome_id)

# Colocando nnome dos suportes
coluna_formatar = ['pokemon1', 'pokemon2', 'pokemon3', 'pokemon4', 'pokemon5']
for coluna in coluna_formatar:
    info_df_exp[coluna] = info_df_exp[coluna].apply(support_id_nome)

# Limpando dados dda tabela
info_df_exp = info_df_exp.drop([0, 1, 2, 3, 4, 5, 'hero_nome', 'nomePc', 'character'], axis=1)

# Adicionando coluna de ordem
info_df_exp['Ordem'] = range(1, len(info_df_exp) + 1)
info_df_exp = info_df_exp[['Ordem', 'name', 'level','endTime', 'pokemon0', 'damage0', 'pokemon1', 'damage1', 'startTime1', 'pokemon2', 'damage2', 'startTime2', 'pokemon3', 'damage3', 'startTime3',  'pokemon4', 'damage4', 'startTime4', 'pokemon5', 'damage5', 'startTime5']]

# Formatar datas
coluna_formatar = ['startTime1', 'startTime2', 'startTime3', 'startTime4', 'startTime5']
for coluna in coluna_formatar:
    info_df_exp[coluna] = pd.to_numeric(info_df_exp[coluna].str.replace(',', '.'))
    info_df_exp[coluna] = pd.to_datetime(info_df_exp[coluna], unit='s').dt.strftime('%M:%S:%f').str[:-3]

info_df_exp.sort_index(ascending=True)


# Renomear colunas
info_df_exp = info_df_exp.rename(columns={'name': 'Jogador', 'endTime': 'Tempo', 'pokemon0': 'Heroi', 'damage0': 'Dano Heroi', 'pokemon1': 'Suporte 1', 'damage1': 'Dano Sup 1', 'startTime1': 'Tempo Sup 1', 'pokemon2': 'Suporte 2', 'damage2': 'Dano Sup 2', 'startTime2': 'Tempo Sup 2', 'pokemon3': 'Suporte 3', 'damage3': 'Dano Sup 3', 'startTime3': 'Tempo Sup 3', 'pokemon4': 'Suporte 4', 'damage4': 'Dano Sup 4', 'startTime4': 'Tempo Sup 4', 'pokemon5': 'Suporte 5', 'damage5': 'Dano Sup 5', 'startTime5': 'Tempo Sup 5'})

info_df_exp

,Ordem,Jogador,level,Tempo,Heroi,Dano Heroi,Suporte 1,Dano Sup 1,Tempo Sup 1,Suporte 2,Dano Sup 2,Tempo Sup 2,Suporte 3,Dano Sup 3,Tempo Sup 3,Suporte 4,Dano Sup 4,Tempo Sup 4,Suporte 5,Dano Sup 5,Tempo Sup 5
0,1,Xanad0,32,21:20.948,Pikachu,252072,Kingler,12869,02:02:755,Arcanine,21683,03:37:872,Umbreon,24401,07:15:975,Beedrill,16531,13:26:362,Electabuzz,9001,16:31:244
1,2,Nico,31,21:48.080,Charmander,199981,Kingler,39379,02:03:868,Arcanine,28350,03:37:493,Vaporeon,25083,07:17:999,Umbreon,16149,13:21:759,Beedrill,20692,16:34:209
2,3,kawa,33,21:53.788,Charmander,195169,Kingler,41197,01:54:023,Arcanine,27260,03:46:239,Vaporeon,27305,07:17:313,Beedrill,20984,13:20:609,Umbreon,20325,16:34:734
3,4,Xanad0,33,21:55.124,Charmander,200447,Umbreon,39939,01:53:977,Kingler,39699,03:40:533,Arcanine,16644,07:31:539,Beedrill,19305,13:26:642,Electabuzz,17250,16:43:868
4,5,Nico,32,21:56.333,Charmander,191340,Meganium,38808,01:58:640,Kingler,30526,03:42:587,Umbreon,32610,07:18:353,Beedrill,22612,13:17:297,Electabuzz,16156,16:33:152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,161,Mestre LpK,32,36:55.494,Charmander,248724,Jolteon,19257,02:17:892,Kingler,57752,05:30:064,Ampharos,27175,11:43:514,Meganium,26668,21:12:032,Arcanine,18627,28:07:744
161,162,Nico,31,37:47.416,Pikachu,317870,Vaporeon,6995,03:43:391,Flareon,7428,08:08:472,Azumarill,11039,15:14:240,Muk,9702,24:51:725,Arcanine,15006,29:32:572
162,163,ackloh mito,36,37:50.273,Bulbasaur,375407,Umbreon,20579,02:34:060,Muk,35382,04:56:057,Alakazam,21825,11:32:812,Azumarill,6588,24:41:343,Pidgeot,4808,29:07:227
163,164,Paula,37,41:37.698,Bulbasaur,478721,Alakazam,30000,02:25:042,Jolteon,24394,07:36:473,Pidgeot,4132,13:28:969,MrMime,0,22:05:949,Espeon,0,28:45:779


## Ranking Unitário

In [756]:
min_indices = info_df_exp.groupby('Jogador')['Tempo'].idxmin()
info_df_exp_un = info_df_exp.loc[min_indices].sort_values(by='Tempo', ascending=True).reset_index(drop=True)
info_df_exp_un['Ordem'] = info_df_exp_un['Ordem'].rank().astype(int).map('{:.0f}°'.format)

info_df_exp_un

,Ordem,Jogador,level,Tempo,Heroi,Dano Heroi,Suporte 1,Dano Sup 1,Tempo Sup 1,Suporte 2,Dano Sup 2,Tempo Sup 2,Suporte 3,Dano Sup 3,Tempo Sup 3,Suporte 4,Dano Sup 4,Tempo Sup 4,Suporte 5,Dano Sup 5,Tempo Sup 5
0,1°,Xanad0,32,21:20.948,Pikachu,252072,Kingler,12869,02:02:755,Arcanine,21683,03:37:872,Umbreon,24401,07:15:975,Beedrill,16531,13:26:362,Electabuzz,9001,16:31:244
1,2°,Nico,31,21:48.080,Charmander,199981,Kingler,39379,02:03:868,Arcanine,28350,03:37:493,Vaporeon,25083,07:17:999,Umbreon,16149,13:21:759,Beedrill,20692,16:34:209
2,3°,kawa,33,21:53.788,Charmander,195169,Kingler,41197,01:54:023,Arcanine,27260,03:46:239,Vaporeon,27305,07:17:313,Beedrill,20984,13:20:609,Umbreon,20325,16:34:734
3,4°,Jumbrado,32,22:04.864,Pikachu,253248,Arcanine,29666,01:59:880,Kingler,15941,03:48:804,Beedrill,27851,07:26:649,Electabuzz,15537,13:26:466,Magneton,12031,17:02:094
4,5°,NewJeans,33,22:10.981,Squirtle,184786,Kingler,42345,02:09:661,Umbreon,40412,03:38:048,Glalie,33945,07:30:903,Magneton,20670,13:28:609,Beedrill,18133,16:48:592
5,6°,Royal,30,22:16.668,Pikachu,259016,Ampharos,22585,02:01:735,Meganium,6962,03:57:838,Victribell,29113,07:42:073,Umbreon,20467,13:46:993,Beedrill,13504,17:21:882
6,7°,Lironi,31,22:53.914,Pikachu,261771,Kingler,37418,01:52:732,Muk,12623,03:41:394,Arcanine,24229,07:23:052,Electabuzz,7284,13:36:978,Magneton,13959,17:02:009
7,8°,Root,30,22:59.368,Charmander,212966,Kingler,46484,01:58:650,Arcanine,31892,03:44:682,Beedrill,23574,07:32:607,Magneton,19096,13:33:098,Electabuzz,17335,17:02:627
8,9°,Weiss,26,23:17.665,Charmander,188407,Ampharos,20356,01:55:362,Muk,38258,04:00:467,Arcanine,21558,08:13:020,Meganium,11057,14:10:607,Electabuzz,18027,17:28:584
9,10°,ftbr,34,23:27.541,Pikachu,277992,Meganium,23724,02:13:320,Espeon,29,04:04:707,Arcanine,20836,08:03:582,Kingler,14462,14:25:616,Beedrill,12821,18:05:028
